In [2]:
!pip install xgboost

In [3]:
import xgboost
import pandas as pd

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,dtype=int)
dataset

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,16884.92400,1,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0
2,28,33.000,3,4449.46200,0,1,1,0
3,33,22.705,0,21984.47061,0,1,1,0
4,32,28.880,0,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0
1334,18,31.920,0,2205.98080,1,0,1,0
1335,18,36.850,0,1629.83350,1,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0


In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [8]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
param_grid = {'n_estimators':[1000], 'max_depth':[7], 'eta':[0.1], 'subsample':[0.7], 'colsample_bytree':[0.8],'use_rmm':["true"],
                    'booster':["gbtree"],'device':["cpu"],
                    'verbosity':[1],'validate_parameters':["false"],'disable_default_eval_metric':["false"],
                    'gamma':[0],'min_child_weight':[1],'max_delta_step':[0],'sampling_method':["uniform"],
                    'colsample_bylevel':[1],'colsample_bynode':[1],'Lambda':[1],'alpha':[1],'tree_method':["auto"],
                    'scale_pos_weight':[1],'refresh_leaf':[1],'process_type':["default"],'grow_policy':["depthwise"],'max_leaves':[0],'max_bin':[256],
                    'num_parallel_tree':[1],'save_period':[0],'task':["train"],'model_in':["NULL"],'model_out':["NULL"],'model_dir':["models/"],'dump_format':["text"],
                   'name_dump':["dump.txt"],'name_pred':["pred.txt"],'pred_margin':[0]}
grid = GridSearchCV(XGBRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1) 

grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\Lenovo\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:29:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "Lambda", "dump_format", "model_dir", "model_in", "model_out", "name_dump", "name_pred", "pred_margin", "save_period", "task" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                         'max_leaves': [0], 'min_child_weight': [1],
                         'model_dir': ['models/'], 'model_in': ['NULL'],
                         'model_out': ['NULL'], 'n_estimators': [1000],
                         'name_dump': ['dump.txt'], 'name_pred': ['pred.txt'],
                         'num_parallel_tree': [1], 'pred_margin': [0],
                         'process_type': ['default'], 'refresh_leaf': [1],
                         'sampling_method': ['uniform'], 'save_period': [0],
                         'scale_pos_weight': [1], ...},
             verbose=3)

In [12]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)

# print classification report
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)

The R_score value for best parameter {'Lambda': 1, 'alpha': 1, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.8, 'device': 'cpu', 'disable_default_eval_metric': 'false', 'dump_format': 'text', 'eta': 0.1, 'gamma': 0, 'grow_policy': 'depthwise', 'max_bin': 256, 'max_delta_step': 0, 'max_depth': 7, 'max_leaves': 0, 'min_child_weight': 1, 'model_dir': 'models/', 'model_in': 'NULL', 'model_out': 'NULL', 'n_estimators': 1000, 'name_dump': 'dump.txt', 'name_pred': 'pred.txt', 'num_parallel_tree': 1, 'pred_margin': 0, 'process_type': 'default', 'refresh_leaf': 1, 'sampling_method': 'uniform', 'save_period': 0, 'scale_pos_weight': 1, 'subsample': 0.7, 'task': 'train', 'tree_method': 'auto', 'use_rmm': 'true', 'validate_parameters': 'false', 'verbosity': 1}: 0.8326083535232259


In [ ]:
table=pd.DataFrame.from_dict(re)

In [ ]:
table

In [ ]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

In [ ]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))